''' this is for prepare training data of identification CNN on oversegmentation and undersegmentation'''

In [7]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import euclidean,cosine
import math
from math import pi
import scipy
from skimage.io import imread
from skimage.measure import label,regionprops
from skimage.color import label2rgb
from skimage import morphology
from os import listdir
from PIL import Image, ImageDraw,ImageFont
from matplotlib import pyplot as plt
from pilutil import toimage
from scipy import ndimage

In [2]:
pos_num='xy01'

dir_path='C:/Users/zoro/Desktop/tracking_work/2018-05-18_HK2_dic_cell_cycle/'+pos_num
ori_img_path=dir_path+'/img'
seg_img_path=dir_path+'/seg'
crop_path=dir_path+'/crop'


ori_img_list=sorted(listdir(ori_img_path))
seg_img_list=sorted(listdir(seg_img_path))
df = pd.read_csv(dir_path + '/Per_Object.csv')
relation_df=pd.read_csv(dir_path + '/Per_Relationships.csv')
am_record=pd.read_csv(dir_path+'/am_record.csv')
t_span=max(df['ImageNumber'])

In [3]:
fuse_pairs=[]
tofuse_cells=[]
fused_cells=[]#include: img_num,obj_num,amount of cells fused in this cell


split_pairs=[]
tosplit_cells=[]##include: img_num,obj_num,amount of cells split from this cell
splited_cells=[]


In [4]:
def compute_overlap_matrix(img_path,img_list,img_num_1,img_num_2):
    frame_1=imread(img_path+'/'+img_list[img_num_1-1]) # mask
    frame_2=imread(img_path+'/'+img_list[img_num_2-1]) # img
    nb_cell_1=np.amax(frame_1)
    nb_cell_2=np.amax(frame_2)

    frame_overlap=np.zeros((nb_cell_1,nb_cell_2))
    for obj_idx1 in range(nb_cell_1):
        obj_num1=obj_idx1+1
        sc_img=(frame_1==obj_num1) # The mask
        ol_judge=np.logical_and(sc_img,frame_2) # croping the cell
        ol_value=np.multiply(ol_judge,frame_2)
        ol_obj2=np.unique(ol_value).tolist()
        #ol_obj2=ol_obj2[ol_obj2!=0]
        ol_obj2.remove(0)
        if len(ol_obj2)>0:
            for obj_num2 in ol_obj2:
                ol_area=np.sum(ol_value==obj_num2)
                obj_idx2=obj_num2-1
                frame_overlap[obj_idx1][obj_idx2]=ol_area  

    return frame_overlap

In [5]:
def generate_single_cell_img(img,seg_img,img_num,obj_num,crop_path):      
    #single_obj_img=morphology.binary_dilation(seg_img==obj_num,morphology.diamond(16))
    single_obj_img=seg_img==obj_num
    single_obj_img=label(single_obj_img)
    rps=regionprops(single_obj_img)
    candi_r=[r for r in rps if r.label==1][0]
    candi_box=candi_r.bbox        
    single_cell_img=single_obj_img*img
    crop_cell_img=single_cell_img[candi_box[0]:candi_box[2],candi_box[1]:candi_box[3]]
#     inds=ndimage.distance_transform_edt(crop_cell_img==0, return_distances=False, return_indices=True)
#     crop_cell_img=crop_cell_img[tuple(inds)]

    cell_img=toimage(crop_cell_img,high=np.max(crop_cell_img),low=np.min(crop_cell_img),mode='I')
    cell_img.save(crop_path+'/i'+str(img_num)+'_o'+str(obj_num)+'.tif')
    
#     re_candi_box=(candi_box[1],candi_box[0],candi_box[3],candi_box[2])
#     candi_obj=pil_img.crop(re_candi_box)
#     candi_obj.save(crop_path+'/box_i'+str(img_num)+'_o'+str(obj_num)+'.tif')


In [8]:
#----------------select obj------------------
img_num_1=127
obj_num=79
img_1=imread(ori_img_path+'/'+ori_img_list[img_num_1-1])
seg_img_1=imread(seg_img_path+'/'+seg_img_list[img_num_1-1])
generate_single_cell_img(img_1,seg_img_1,img_num_1,obj_num,crop_path)
# crop_img=imread(crop_path+'/i1_o5.tif')
# plt.imshow(crop_img)
# plt.show()

c:\users\zoro\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  app.launch_new_instance()


In [127]:
#----------------single cell------------------
img_num_1=45
img_1=imread(ori_img_path+'/'+ori_img_list[img_num_1-1])
seg_img_1=imread(seg_img_path+'/'+seg_img_list[img_num_1-1])
max_obj_num=np.amax(seg_img_1)
for obj_num in range(1,max_obj_num+1):
    generate_single_cell_img(img_1,seg_img_1,img_num_1,obj_num,crop_path)

c:\users\zoro\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  


In [8]:
i_n=29
for img_num in range(i_n,i_n+1):
    if img_num==t_span:
        break
    img_num_1=img_num
    img_num_2=img_num+1
    img_1=imread(ori_img_path+'/'+ori_img_list[img_num_1-1])
    img_2=imread(ori_img_path+'/'+ori_img_list[img_num_2-1])
    pil_img_1=Image.open(ori_img_path+'/'+ori_img_list[img_num_1-1])
    pil_img_2=Image.open(ori_img_path+'/'+ori_img_list[img_num_2-1])
    
    seg_img_1=imread(seg_img_path+'/'+seg_img_list[img_num_1-1])
    seg_img_2=imread(seg_img_path+'/'+seg_img_list[img_num_2-1])
    
    
    frame_overlap=compute_overlap_matrix(seg_img_path,seg_img_list,img_num_1,img_num_2)
    area_arr=df.loc[(df['ImageNumber']==img_num_1),'Cell_AreaShape_Area'].values
    area_arr_R=df.loc[(df['ImageNumber']==img_num_2),'Cell_AreaShape_Area'].values
    

    
    nb_cell_1=frame_overlap.shape[0]
    nb_cell_2=frame_overlap.shape[1]
    
    #-------calculate the cell merge --------------
    frame_fusion = np.zeros(frame_overlap.shape)
    for source_o_n in range(1,nb_cell_1+1):
        ol_target=frame_overlap[source_o_n-1,:]
        if np.all(ol_target==0):#if source obj have no overlap target
            target_o_n=0    
        else:      
            target_o_n=np.argmax(frame_overlap,axis=1)[source_o_n-1]+1
        
       
        if target_o_n> 0:
            frame_fusion[source_o_n-1, target_o_n-1] = 1
    
        

    # remove any columns with a single lingering value
    S = np.sum(frame_fusion>0,axis=0)
    frame_fusion[:, S==1] = 0
    
        
    # Update the sum vector
    S = np.sum(frame_fusion, axis=0)
    


    #Find the fused cell regions number
#    fused_cells = np.where(S >= 2)[0]+1
#     print(fused_cells)
    for i in range(len(np.where(S >= 2)[0])):
        print(img_num_2,np.where(S >= 2)[0][i]+1,'fused')
        fused_cells.append([img_num_2,np.where(S >= 2)[0][i]+1,S[np.where(S >= 2)[0][i]]])
        generate_single_cell_img(img_2,seg_img_2,img_num_2,np.where(S >= 2)[0][i]+1,crop_path)
    for i in range(len(np.where(frame_fusion==1)[0])):
        fused_pairs.append([img_num_1,np.where(frame_fusion==1)[0][i]+1,img_num_2,np.where(frame_fusion==1)[1][i]+1])
        tofuse_cells.append([img_num_1,np.where(frame_fusion==1)[0][i]+1])
        print(img_num_1,np.where(frame_fusion==1)[0][i]+1,'to_fuse')
        generate_single_cell_img(img_1,seg_img_1,img_num_1,np.where(frame_fusion==1)[0][i]+1,crop_path)
    
    #print(fused_pairs)
    
#     #If we didn't find any fusion cases, return
#     if np.all(S < 2):
#         continue
#------------------------------calculate cell split --------------------------
    frame_overlap_R=np.transpose(frame_overlap)
    frame_split = np.zeros(frame_overlap_R.shape)
    for source_R_o_n in range(1,nb_cell_2+1):
        ol_target_R=frame_overlap_R[source_R_o_n-1,:]
        if np.all(ol_target_R==0):#if source_R obj have no overlap target_R
            target_R_o_n=0    
        else:      
            target_R_o_n=np.argmax(frame_overlap_R,axis=1)[source_R_o_n-1]+1
        
       
        if target_R_o_n> 0:
            frame_split[source_R_o_n-1, target_R_o_n-1] = 1
    
          

    # remove any columns with a single lingering value
    S = np.sum(frame_split>0,axis=0)
    frame_split[:, S==1] = 0
    
    # Update the sum vector
    S = np.sum(frame_split, axis=0)



    #Find the fused cell regions number
#     split_cells = np.where(S >= 2)[0]+1
#     print(split_cells)
    for i in range(len(np.where(S >= 2)[0])):
        tosplit_cells.append([img_num_1,np.where(S >= 2)[0][i]+1,S[np.where(S >= 2)[0][i]]])
        print(img_num_1,np.where(S >= 2)[0][i]+1,'tosplit')
        generate_single_cell_img(img_1,seg_img_1,img_num_1,np.where(S >= 2)[0][i]+1,crop_path)
    for i in range(len(np.where(frame_split==1)[0])):
        split_pairs.append([img_num_1,np.where(frame_split==1)[1][i]+1,img_num_2,np.where(frame_split==1)[0][i]+1])
        splited_cells.append([img_num_2,np.where(frame_split==1)[0][i]+1])
        print(img_num_2,np.where(frame_split==1)[0][i]+1,'splited')
        generate_single_cell_img(img_2,seg_img_2,img_num_2,np.where(frame_split==1)[0][i]+1,crop_path)
    #print(split_pairs)
    
#     #If we didn't find any split cases, return
#     if np.all(S < 2):
#         continue
